In [2]:
import pandas as pd
df = pd.read_csv('../../Output/output_1.csv')
df.drop(columns=[x for x in df.columns if 'id' in x])

,x,y,z,type,1-neighbour type,1-neighbour x,1-neighbour y,1-neighbour z,2-neighbour type,2-neighbour x,...,4-faced,5-faced,6-faced,7-faced,8-faced,9-faced,10-faced,11-faced,12-faced,13-faced
0,26.0291,54.92020,24.45400,1.28,1.28,26.7424,53.88120,22.42570,1.60,23.7794,...,1.0,3.0,4.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0
1,42.4982,36.78260,37.07070,1.60,1.28,44.2310,38.35390,35.84600,1.28,40.5901,...,0.0,1.0,10.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2,44.3755,24.90850,17.91550,1.60,1.28,43.8712,22.74440,16.34050,1.28,41.8245,...,1.0,1.0,9.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0
3,53.9179,13.26660,54.69820,1.28,1.28,51.7705,14.43000,53.93900,1.28,52.5371,...,1.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.7815,28.60990,25.90210,1.28,1.28,31.8357,30.59160,25.31700,1.28,28.2009,...,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,26.5783,7.06695,20.12560,1.28,1.28,28.5276,8.42626,20.08580,1.28,24.2580,...,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13496,54.6684,48.03910,53.69850,1.28,1.28,53.4106,49.75090,52.50440,1.28,54.4029,...,0.0,5.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
13497,35.8328,47.42480,6.19108,1.60,1.28,35.0159,46.58650,3.81853,1.28,33.2776,...,0.0,2.0,8.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
13498,15.7849,50.60780,10.77430,1.28,1.28,15.7732,52.02770,12.91640,1.28,15.9683,...,2.0,2.0,4.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [4]:
data = df.to_numpy()
data = data.astype(np.float32)

inp_data_train, inp_data_test, out_data_train, out_data_test = train_test_split(
    data[:, :-14], data[:, -14:], test_size=0.2, random_state=42)

inp_data_train, out_data_train = \
    torch.from_numpy(inp_data_train), torch.from_numpy(out_data_train)
inp_data_test, out_data_test = \
    torch.from_numpy(inp_data_test), torch.from_numpy(out_data_test)

In [5]:
train_dataset = TensorDataset(inp_data_train, out_data_train)
test_dataset = TensorDataset(inp_data_test, out_data_test)

In [6]:
BATCH_SIZE = 256

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [8]:
import torch.nn as nn
import torch.nn.functional as nf

class RegressionMultiout(nn.Module):
    def __init__(self, inputs_size, hidden_size, output_size):
        super(RegressionMultiout, self).__init__()
        
        self.vorono = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(inputs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )
        
        self.init_weights()
        
    def init_weights(self):
        
        def inner_init_weights(layer):
            if not isinstance(layer, nn.Linear):
                return
            
            nn.init.xavier_uniform_(layer.weight.data)
            nn.init.constant_(layer.bias.data, 0.1)
            
        self.vorono.apply(inner_init_weights)

        
    def forward(self, x):
        return self.vorono(x)

In [9]:
INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE = inp_data_train.shape[1], 20, out_data_train.shape[1]

In [10]:
model = RegressionMultiout(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
model

RegressionMultiout(
  (vorono): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=105, out_features=20, bias=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=20, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): ReLU()
    (8): Linear(in_features=20, out_features=14, bias=True)
  )
)

In [11]:
num_epochs = 1_000
batch_size = len(train_loader)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)

In [17]:
import matplotlib.pyplot as plt

train_loss = []
outputs = {}

for epoch in range(1, num_epochs + 1):
    
    running_loss = 0
    
    for inp, out in train_loader:
        
        prediction = model(inp)
        loss = criterion(prediction, out)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    scheduler.step()
    
    running_loss /= batch_size
    train_loss.append(running_loss)
    outputs[epoch] = {'img':inp, 'out':prediction}
    
    if epoch % 10 == 0:
        print('%03d.learning_rate: %.05f' % (epoch, optimizer.param_groups[0]["lr"]))
        print('%03d.running_loss: %.05f' % (epoch, running_loss))
    
plt.plot(range(1, num_epochs - 9), train_loss[10:])
plt.xlabel('Number of epochs')
plt.ylabel('Training Loss...')
plt.show()

010.learning_rate: 0.00174
010.running_loss: 1.24099
020.learning_rate: 0.00174
020.running_loss: 1.24099
030.learning_rate: 0.00174
030.running_loss: 1.24099
040.learning_rate: 0.00174
040.running_loss: 1.24099
050.learning_rate: 0.00174
050.running_loss: 1.24099
060.learning_rate: 0.00174
060.running_loss: 1.24099
070.learning_rate: 0.00174
070.running_loss: 1.24099
080.learning_rate: 0.00174
080.running_loss: 1.24098
090.learning_rate: 0.00174
090.running_loss: 1.24098
100.learning_rate: 0.00157
100.running_loss: 1.24098
110.learning_rate: 0.00157
110.running_loss: 1.24096
120.learning_rate: 0.00157
120.running_loss: 1.24096
130.learning_rate: 0.00157
130.running_loss: 1.24096
140.learning_rate: 0.00157
140.running_loss: 1.24096
150.learning_rate: 0.00157
150.running_loss: 1.24096
160.learning_rate: 0.00157
160.running_loss: 1.24095
170.learning_rate: 0.00157
170.running_loss: 1.24095
180.learning_rate: 0.00157
180.running_loss: 1.24095
190.learning_rate: 0.00157
190.running_loss: 1

KeyboardInterrupt: 

In [13]:
predictions = model(inp_data_test)

In [14]:
idx = np.random.randint(0, predictions.shape[0])
print('Prediction :', np.round(predictions[idx].detach().numpy(), 0))
print('Real output:', out_data_test[idx].detach().numpy())

Prediction : [13.  0. -0.  0.  0.  2.  7.  3.  0.  0.  0.  0. -0.  0.]
Real output: [13.  0.  0.  0.  2.  2.  4.  3.  2.  0.  0.  0.  0.  0.]


In [15]:
predictions = model(inp_data_train)

In [16]:
idx = np.random.randint(0, predictions.shape[0])
print('Prediction :', np.round(predictions[idx].detach().numpy(), 0))
print('Real output:', out_data_train[idx].detach().numpy())

Prediction : [13.  0. -0.  0.  0.  2.  7.  3.  0.  0.  0.  0. -0.  0.]
Real output: [13.  0.  0.  0.  0.  1. 10.  2.  0.  0.  0.  0.  0.  0.]
